In [1]:
Link = 'https://github.com/natsunoyuki/blog_posts/blob/main/data_science/Bayesian%20Optimization%20of%20Model%20Hyperparameters.ipynb'

In [2]:
pip install niapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 181 kB 14.2 MB/s 


In [3]:
pip install sklearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=04717d792699307662b57420d027b7fed5c6d6464917acb2060eae2e35f3c556
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [4]:
pip install sklearn-nature-inspired-algorithms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 175 kB 80.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: niapy
    Found existing installation: niapy 2.0.4
    Uninstalling niapy-2.0.4:
      Successfully uninstalled niapy-2.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [5]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
SEED = 1412
Test_Ratio = 0.2

In [7]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X,y = make_classification(n_samples=10000,n_features=5,n_informative=2,n_classes=2,n_clusters_per_class=1,flip_y=0.1412,shuffle=False,random_state=SEED)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = Test_Ratio, random_state = SEED)



In [21]:
import json
import numpy as np 
from niapy.algorithms.basic import CuckooSearch, FireflyAlgorithm, BatAlgorithm
from niapy.algorithms.modified import HybridBatAlgorithm, AdaptiveBatAlgorithm, HybridSelfAdaptiveBatAlgorithm

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier


SCORING FUNCTIONS (6)

CUCKOO SEARCH

In [22]:
CucKoo_pbounds = {'population_size':(10,100),'pa':(0.1,1.0) ,'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Cuckoo_dict = dict()

def Save_cuckoo_dict():
    with open('Cuckoo_results.json','w') as dict_dump:
        json.dump(Cuckoo_dict,dict_dump)

def Score_Cuckoo(population_size,pa,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{pa}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in Cuckoo_dict:
        return Cuckoo_dict[combo_str]

    Algo = CuckooSearch()
    Algo.set_parameters(population_size=population_size,pa=pa,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    Cuckoo_dict[combo_str] = val 

    return val


CUCKOO SEARCH

FIRE FLY 

In [23]:
FireFly_pbounds = {'population_size':(10,100),'alpha':(0.1,1.0),'beta0':(0.01,100),'gamma':(0.1,1.0),'theta':(0.1,1.0),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Firefly_dict = dict()

def Save_firefly_dict():
    with open('Firefly_results.json','w') as dict_dump:
        json.dump(Firefly_dict,dict_dump)

def Score_Firefly(population_size,alpha,beta0,gamma,theta,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{alpha}_{beta0}_{gamma}_{theta}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in Firefly_dict:
        return Firefly_dict[combo_str]

    Algo = FireflyAlgorithm()
    Algo.set_parameters(population_size=population_size,alpha=alpha,beta0=beta0,gamma=gamma,theta=theta,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    Firefly_dict[combo_str] = val 

    return val


FIRE FLY 

BAT

In [24]:

BAT_pbounds = {'population_size':(10,100),'loudness':(0.1,1.0),'pulse_rate':(0.1,1.0),'gamma':(0.1,1.0),'alpha':(0.1,1.0),'min_frequency':(0,10),'max_frequency':(10,100),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Bat_dict = dict()

def Save_bat_dict():
    with open('Bat_results.json','w') as dict_dump:
        json.dump(Bat_dict,dict_dump)

def Score_Bat(population_size,loudness,pulse_rate,gamma,alpha,min_frequency,max_frequency,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{loudness}_{pulse_rate}_{gamma}_{alpha}_{min_frequency}_{max_frequency}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in Bat_dict:
        return Bat_dict[combo_str]

    Algo = BatAlgorithm()
    Algo.set_parameters(population_size=population_size,loudness=loudness,pulse_rate=pulse_rate,alpha=alpha,gamma=gamma,min_frequency=min_frequency,max_frequency=max_frequency,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    Bat_dict[combo_str] = val 

    return val


BAT

HYBRID BAT

In [25]:
HBA_pbounds = {'differential_weight':(0.1,1.0),'crossover_probability':(0.1,1.0),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}


HBA_dict = dict()

def Save_hba_dict():
    with open('HBA_results.json','w') as dict_dump:
        json.dump(HBA_dict,dict_dump)

def Score_HBA(differential_weight,crossover_probability,n_estimator,criterion,max_feature):
    
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{differential_weight}_{crossover_probability}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in HBA_dict:
        return HBA_dict[combo_str]

    Algo = HybridBatAlgorithm()
    Algo.set_parameters(differential_weight=differential_weight,crossover_probability=crossover_probability,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    HBA_dict[combo_str] = val 

    return val


HYBRID BAT

SELF ADAPTIVE BAT

In [26]:
SABA_pbounds = {'population_size':(10,100),'loudness':(0.1,1.0),'pulse_rate':(0.1,1.0),'epsilon':(0,0.1),'alpha':(0.1,1.0),'min_frequency':(0,10),'max_frequency':(10,100),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

SBA_dict = dict()

def Save_sba_dict():
    with open('SBA_results.json','w') as dict_dump:
        json.dump(SBA_dict,dict_dump)

def Score_SBA(population_size,loudness,pulse_rate,epsilon,alpha,min_frequency,max_frequency,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_{loudness}_{pulse_rate}_{epsilon}_{alpha}_{min_frequency}_{max_frequency}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in SBA_dict:
        return SBA_dict[combo_str]

    Algo = AdaptiveBatAlgorithm()
    Algo.set_parameters(population_size=population_size,loudness=loudness,pulse_rate=pulse_rate,alpha=alpha,epsilon=epsilon,min_frequency=min_frequency,max_frequency=max_frequency,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    SBA_dict[combo_str] = val 

    return val


SELF ADAPTIVE BAT

HYBRID SELF ADAPTIVE BAT

In [27]:
HSABA_pbounds = {'differential_weight':(0.1,1.0),'crossover_probability':(0.1,1.0),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

HSABA_dict = dict()

def Save_hsaba_dict():
    with open('HSABA_results.json','w') as dict_dump:
        json.dump(HSABA_dict,dict_dump)

def Score_HBA(differential_weight,crossover_probability,n_estimator,criterion,max_feature):
    
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{differential_weight}_{crossover_probability}_{n_estimator}_{criterion}_{max_feature}'

    if combo_str in HSABA_dict:
        return HSABA_dict[combo_str]

    Algo = HybridSelfAdaptiveBatAlgorithm()
    Algo.set_parameters(differential_weight=differential_weight,crossover_probability=crossover_probability,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    nia_mdl.fit(X_train,y_train)
    val = nia_mdl.score(X_test,y_test)
    HSABA_dict[combo_str] = val 

    return val


HYBRID SELF ADAPTIVE BAT

SCORING FUNCTIONS (6)

FUNCTION LIST

In [28]:
NIA_Val_Dict = {0:'Cuckoo Search',1:'Fire Fly',2:'Bat',3:'Hybrid Bat',4:'Self Adaptive Bat',5:'Hybrid Self Adaptive Bat'}

NIA_lst = [CucKoo_pbounds, FireFly_pbounds, BAT_pbounds, HBA_pbounds, SABA_pbounds, HSABA_pbounds]

Save_dict_fxn = [Save_cuckoo_dict, Save_firefly_dict, Save_bat_dict, Save_hba_dict, Save_sba_dict, Save_hsaba_dict]

NIA_Score_fxn = [Score_Cuckoo, Score_Firefly, Score_Bat, Score_HBA, Score_SBA, Score_HBA]

FUNCTION LIST

BASE CASE

In [29]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
val = clf.score(X_test,y_test)
print(f'The default value is -> {val}')

The default value is -> 0.8975


BASE CASE

In [30]:
from bayes_opt import BayesianOptimization, UtilityFunction
import matplotlib.pyplot as plt


In [31]:
def Optimize_and_plot(bounds,score_fxn,Algo_name:str,Iters:int,Dict_save_fxn):

    optimizer = BayesianOptimization(f = None, pbounds = bounds, verbose = 2, random_state = SEED)
    utility = UtilityFunction(kind = "ucb", kappa = 1.96, xi = 0.01)

    for i in range(Iters):
        # Get optimizer to suggest a new parameter value to try.
        next_point = optimizer.suggest(utility)  
        # Evaluate the output of the black_box_function using the new parameter value.
        target = score_fxn(**next_point)
        try:
            # Update the optimizer with the evaluation results. This needs to be in try-except
            # to prevent repeat errors from occuring.
            optimizer.register(params = next_point, target = target)
        except:
            pass
    
    print(f'Best result: {optimizer.max["params"]}; f(x) = {optimizer.max["target"]}.')

    Dict_save_fxn()
    

    plt.plot(range(1, 1+len(optimizer.space.target)), optimizer.space.target, "-o")
    plt.grid(True)
    plt.xlabel("Iteration")
    plt.ylabel("Score")
    plt.legend()
    plt.savefig(f'{Algo_name}_{Iters}_Iterations.svg',format='svg')
    plt.show()

In [32]:
NIA_Val_Dict = {0:'Cuckoo Search',1:'Fire Fly',2:'Bat',3:'Hybrid Bat',4:'Self Adaptive Bat',5:'Hybrid Self Adaptive Bat'}

NIA_lst = [CucKoo_pbounds, FireFly_pbounds, BAT_pbounds, HBA_pbounds, SABA_pbounds, HSABA_pbounds]

Save_dict_fxn = [Save_cuckoo_dict, Save_firefly_dict, Save_bat_dict, Save_hba_dict, Save_sba_dict, Save_hsaba_dict]

NIA_Score_fxn = [Score_Cuckoo, Score_Firefly, Score_Bat, Score_HBA, Score_SBA, Score_HBA]

In [ ]:
for i in range(len(NIA_lst)):
    Optimize_and_plot(bounds=NIA_lst[i],score_fxn=NIA_Score_fxn[i],Algo_name=NIA_Val_Dict[i],Dict_save_fxn=Save_dict_fxn[i],Iters=100)
    

Fitting at most 1.0 candidates


function ClickConnect(){

console.log("Working"); 
document.querySelector("#comments > span").click() 
}
setInterval(ClickConnect,5000)

Basement Don't go down

In [ ]:
break

In [ ]:
# Set range of C to optimize over. bayes_opt requires this to be a dictionary.
pbounds = {"algo_val": (0, len(HBA_lst)-1), "mdl_val" : (0,len(mdl_para_lst)-1)}
print(pbounds)
#ptypes = {"algo_val": int, "mdl_val": int}

# Create BayesianOptimization object, and optimize (maximize) black_box_function.
optimizer = BayesianOptimization(f = Get_Score_HBA, pbounds = pbounds, verbose = 2, random_state = SEED)

optimizer.maximize(init_points = 50, n_iter = 50)

print(f'Best result: {optimizer.max["params"]} f(x) = {optimizer.max["target"]}.')

In [ ]:
# Create optimizer and utility function objects.
optimizer = BayesianOptimization(f = None, pbounds = {"algo_val": (0, len(HBA_lst)-1), "mdl_val" : (0,len(mdl_para_lst)-1)}, verbose = 2, random_state = SEED)
utility = UtilityFunction(kind = "ucb", kappa = 1.96, xi = 0.01)

# Optimization for loop.
for i in range(100):
    # Get optimizer to suggest a new parameter value to try.
    next_point = optimizer.suggest(utility)  
    # Evaluate the output of the black_box_function using the new parameter value.
    target = Get_Score_HBA(**next_point)
    try:
        # Update the optimizer with the evaluation results. This needs to be in try-except
        # to prevent repeat errors from occuring.
        optimizer.register(params = next_point, target = target)
    except:
        pass
    
print(f'Best result: {optimizer.max["params"]}; f(x) = {optimizer.max["target"]}.' )

In [ ]:

#plt.figure(figsize = (15, 5))
plt.plot(range(1, 1+len(optimizer.space.target)), optimizer.space.target, "-o")
plt.grid(True)
plt.xlabel("Iteration")
plt.ylabel("Black box function")
plt.legend()
plt.show()